In [ ]:
from sympy import *
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

from scipy.interpolate import RegularGridInterpolator
from pathlib import Path

In [ ]:
# Ansazte
q = symbols('q', real=True, positive=True)
A0, r, x, y, n0 = symbols('A_0 r x y n_0', real=True)

n_T = n0 + 2*A0*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))
n_S = n0 + 2*A0*(cos(q*y))
n_H = n0 - 2*A0*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))

# n_T = n0 + 2*n0/3*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))
# n_S = n0 + 2*n0/2*(cos(q*y))
# n_H = n0 - 2*n0/6*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))

# n_T = n0 + 2*n0/3.2*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))
# n_S = n0 + 2*n0/2.2*(cos(q*y))
# n_H = n0 - 2*n0/6.2*(cos(q*Rational(1,2)*(-sqrt(3)*x - y)) + cos(q*y) + cos(q*Rational(1,2)*(sqrt(3)*x - y)))

# numpy lambdas
n_L_np = lambda A0, x, y, n0: n0 * np.ones_like(x)
n_T_np = lambdify((A0, x, y, n0), n_T.subs(q, 1), 'numpy')
n_S_np = lambdify((A0, x, y, n0), n_S.subs(q, 1), 'numpy')
n_H_np = lambdify((A0, x, y, n0), n_H.subs(q, 1), 'numpy')

In [ ]:
# Plotting test, plot subfigures for each ansazte in plotly
x_vals = np.linspace(0, 1/np.sqrt(3) * 4*np.pi, 100, endpoint=False)
y_vals = np.linspace(0, 4*np.pi, 180, endpoint=False)
X, Y = np.meshgrid(x_vals, y_vals)

dy, dx = Y[1,0], X[0,1]
ky = np.fft.fftfreq(len(y_vals), d=dy) * 2 * np.pi
kx = np.fft.fftfreq(len(x_vals), d=dx) * 2 * np.pi
kx, ky = np.meshgrid(kx, ky)
k2 = kx**2 + ky**2
k4 = k2**2
k6 = k2**3

In [ ]:
def f_np(n, q0, beta, epsilon, g):
    n_k = np.fft.fft2(n)
    f_lin = np.fft.ifft2(beta * (k4 - 2 * k2 * q0**2) / 2 * n_k).real * n

    f_ln = np.ones_like(n)*100
    mask_pos = (n > 0)
    f_ln[mask_pos] = (n[mask_pos]) * np.log(n[mask_pos])
    f_ln[~mask_pos] *= -n[~mask_pos]  # Artificially n*log(n) -> -100*n as n -> 0 from negative side

    f_nl = f_ln + n**3 * g / 3 + n**4 / 4 + n**2 * (epsilon + beta*q0**2) / 2
    # f_nl = n**3 * g / 3 + n**4 / 4 + n**2 * (epsilon + beta*q0**2) / 2

    return np.sum((f_lin + f_nl))/n.size
f_np(n_T_np(1./3, X, Y, 1.), 1, 0.289, 0.2, -0.5)

In [ ]:
def trim_to_nonzero_region(arr, pad=50):
    """
    Keep values from just before the first nonzero to just after the last nonzero,
    padding by `pad` indices on each side. Everything else is set to NaN.

    Parameters
    ----------
    arr : np.ndarray
        1D array to process.
    pad : int, optional
        Number of indices to include before the first nonzero and after the last nonzero.

    Returns
    -------
    trimmed : np.ndarray
        Copy of arr with values outside the region set to NaN.
    """
    arr = arr.copy()
    nz = np.flatnonzero(arr)

    if nz.size > 0:
        first = max(nz[0] - pad, 0)
        last  = min(nz[-1] + pad, arr.size - 1)

        mask = np.ones_like(arr, dtype=bool)
        mask[first:last+1] = False
        arr[mask] = np.nan
    else:
        arr[:] = np.nan

    return arr


In [ ]:
def get_fLeastLTSH(n0, beta, epsilon, g):
  _epsilonRange = params["epsilonRange"]
  _n0Range = params["n0Range"]

  _fL_np = np.zeros_like(n0)
  _fT_np = np.zeros_like(n0)
  _fS_np = np.zeros_like(n0)
  _fH_np = np.zeros_like(n0)
  # _A00Range = np.linspace(0.01, 1.0, 10, endpoint=True)
  interpA00T = RegularGridInterpolator((_epsilonRange, _n0Range), A00Tmin)
  interpA00S = RegularGridInterpolator((_epsilonRange, _n0Range), A00Smin)
  interpA00H = RegularGridInterpolator((_epsilonRange, _n0Range), A00Hmin)
  _A00Tmins = interpA00T((epsilon, n0))
  _A00Smins = interpA00S((epsilon, n0))
  _A00Hmins = interpA00H((epsilon, n0))

  _A00Tmins = trim_to_nonzero_region(_A00Tmins)
  _A00Smins = trim_to_nonzero_region(_A00Smins)
  _A00Hmins = trim_to_nonzero_region(_A00Hmins)

  for _i, _n0 in enumerate(n0):
    # _fAT = np.array([f_np(n_T_np(A00*_n0/3, X, Y, _n0), 1, beta, epsilon, g) for A00 in _A00Range])
    # _A00Tmin, _ = interpolate_min(_A00Range, _fAT)
    # _fAS = np.array([f_np(n_S_np(A00*_n0/2, X, Y, _n0), 1, beta, epsilon, g) for A00 in _A00Range])
    # _A00Smin, _ = interpolate_min(_A00Range, _fAS)
    # _fAH = np.array([f_np(n_H_np(A00*_n0/6, X, Y, _n0), 1, beta, epsilon, g) for A00 in _A00Range])
    # _A00Hmin, _ = interpolate_min(_A00Range, _fAH)
    _A00Tmin = _A00Tmins[_i]
    _A00Smin = _A00Smins[_i]
    _A00Hmin = _A00Hmins[_i]
    # _A00Hmin = interpA00H((epsilon, _n0))
    
    _fL_np[_i] = f_np(n_L_np(0, X, Y, _n0), 1, beta, epsilon, g)-1e-12  # avoid exact equality issues
    _fT_np[_i] = f_np(n_T_np(_A00Tmin *_n0/3, X, Y, _n0), 1, beta, epsilon, g)
    _fS_np[_i] = f_np(n_S_np(_A00Smin *_n0/2, X, Y, _n0), 1, beta, epsilon, g)
    _fH_np[_i] = f_np(n_H_np(_A00Hmin *_n0/6, X, Y, _n0), 1, beta, epsilon, g)

  # stack into (4, N)
  all_vals = np.stack([_fL_np, _fT_np, _fS_np, _fH_np], axis=0)

  # compute min ignoring NaNs
  min_vals = np.nanmin(all_vals, axis=0)

  # masks: True where candidate equals min and is not NaN
  _fL_least = (_fL_np    == min_vals) & ~np.isnan(_fL_np)
  _fT_least = (_fT_np == min_vals) & ~_fL_least
  _fS_least = (_fS_np == min_vals) & ~_fL_least
  _fH_least = (_fH_np == min_vals) & ~_fL_least
  
  return (_fL_np,
          _fT_np,
          _fS_np,
          _fH_np,
          _fL_least,
          _fT_least,
          _fS_least,
          _fH_least)


In [ ]:
def findIntersect(n, f1, f2):
  """
  Find intersection points between f1(n) and f2(n).
  
  Parameters
  ----------
  n : np.ndarray
      1D array of x-values (must be sorted).
  f1, f2 : np.ndarray
      1D arrays of function values at n. May contain NaNs.
  
  Returns
  -------
  intersections : np.ndarray
      Array of n-values where f1 and f2 intersect (linear interpolation).
  """
  # Mask out NaNs
  mask = ~np.isnan(f1) & ~np.isnan(f2)
  n_valid = n[mask]
  f1_valid = f1[mask]
  f2_valid = f2[mask]

  # Difference
  diff = f1_valid - f2_valid

  # Find sign changes
  sign_changes = np.where(np.sign(diff[:-1]) * np.sign(diff[1:]) < 0)[0]

  # Interpolate crossing points
  intersections = []
  for i in sign_changes:
      n0, n1 = n_valid[i], n_valid[i+1]
      d0, d1 = diff[i], diff[i+1]
      f0, f1 = f1_valid[i], f1_valid[i+1]
      # Linear interpolation for root of diff
      n_cross = n0 - d0 * (n1 - n0) / (d1 - d0)
      f_cross = f0 + (f1 - f0) * (n_cross - n0) / (n1 - n0)
      intersections.append((n_cross, f_cross))

  if len(intersections) == 0:
      return [np.array((np.nan, np.nan))]
  return np.array(intersections)

In [ ]:
# list .npz files in ./out/prospectus
import os

def list_npz_files(directory, startswith='Log_'):
    return [f for f in os.listdir(directory) if f.endswith('.npz') and f.startswith(startswith)]

LogFiles = list_npz_files('./out/prospectus', 'Log_')
StdFiles = list_npz_files('./out/prospectus', 'Std_')
LogFiles, StdFiles

In [ ]:
file = 'Log_PhaseDiagram_g-0.5.npz'
data = np.load('./out/prospectus/' + file)
if 'g' in data.keys():
  g_log = data['g']
  beta_log = data['beta']
  epsilonCrit_log = data['epsilonCrit']
  n0Crit_log = data['n0Crit']
else:
  g_log = np.nan
  epsilonCrit_log = np.nan
  n0Crit_log = np.nan
epsilonRange_log = data['epsilonRange']
n0Range_log = data['n0Range']
n_LT_log = data['n_LT']
n_TS_log = data['n_TS']
n_SH_log = data['n_SH']
n_HL_log = data['n_HL']
if 'A00Tmin' in data.keys():
  A00Tmin = data['A00Tmin']
  A00Smin = data['A00Smin']
  A00Hmin = data['A00Hmin']
else:
  A00Tmin = None
  A00Smin = None
  A00Hmin = None

In [ ]:
params = {"n0Range": n0Range_log,
          "epsilonRange": epsilonRange_log,
         }
_epsilon = epsilonRange_log[int(len(epsilonRange_log)/2)]
( _fL_np,
  _fTpos_np,
  _fSpos_np,
  _fHpos_np,
  _fL_least,
  _fT_least,
  _fS_least,
  _fH_least ) = get_fLeastLTSH(params["n0Range"], beta_log, _epsilon, g_log)

In [ ]:
nf_LT = findIntersect(params["n0Range"], _fL_np, _fTpos_np)[0]
nf_TS = findIntersect(params["n0Range"][~_fL_least], _fTpos_np[~_fL_least], _fSpos_np[~_fL_least])[0]
nf_SH = findIntersect(params["n0Range"][~_fL_least&~_fT_least], _fSpos_np[~_fL_least&~_fT_least], _fHpos_np[~_fL_least&~_fT_least])[0]
nf_HL = findIntersect(params["n0Range"], _fHpos_np, _fL_np)[-1]

(nf_LT, nf_TS, nf_SH, nf_HL)

In [ ]:
def add_sparse_markers(fig, x, y, start=0, step=10, 
                       marker_symbol='circle', 
                       color='blue', 
                       size=8, 
                       name='',
                       showlegend=True,
                       opacity=1.0):
    """
    Add a line with sparse markers and a unified legend entry.

    Parameters
    ----------
    fig : plotly.graph_objects.Figure
        The figure to add traces to.
    x, y : array-like
        Data arrays.
    step : int, optional
        Interval for marker subsampling (default 10).
    marker_symbol : str, optional
        Plotly marker symbol (default 'circle').
    color : str, optional
        Line/marker color (default 'blue').
    size : int, optional
        Marker size (default 8).
    name : str, optional
        Legend name (default 'trace').
    """
    # Remove NaNs
    # mask_valid = ~np.isnan(x) & ~np.isnan(y)
    # x = x[mask_valid]
    # y = y[mask_valid]
    if start==0:
      start=step

    # Line only (no legend)
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color=color),
        showlegend=False,
        opacity=opacity
    ))

    # Sparse markers (no legend)
    fig.add_trace(go.Scatter(
        x=x[(step-start)::step],
        y=y[(step-start)::step],
        mode='markers',
        marker_symbol=marker_symbol,
        marker=dict(color=color, size=size),
        showlegend=False,
        hoverinfo='skip',       # ignore hover
        opacity=opacity
    ))

    # Legend glyph trace (just two points so legend shows line+marker)
    fig.add_trace(go.Scatter(
        x=[x[0], x[1]],
        y=[y[0], y[1]],
        mode='lines+markers',
        line=dict(color=color),
        marker_symbol=marker_symbol,
        marker=dict(color=color, size=size),
        name=name,
        hoverinfo='skip',       # ignore hover
        showlegend=showlegend,
        opacity=opacity
    ))


In [ ]:
_n_min = nf_LT[0] - (nf_HL[0] - nf_LT[0])*0.3
_n_max = nf_HL[0] + (nf_HL[0] - nf_LT[0])*0.3

In [ ]:
# slope adjustment:
_n0Range = params["n0Range"]
_inRange = (_n0Range > _n_min) & (_n0Range < _n_max)

n0_mid = (nf_HL[0] + nf_LT[0])/2
m = (nf_HL[1] - nf_LT[1])/(nf_HL[0] - nf_LT[0])

_f_max = np.nanmax(np.where(_fL_least, _fL_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid))

fig = go.Figure()
add_sparse_markers(fig, _n0Range[_inRange], np.where(_fL_least, _fL_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='line-ns',
                   color='red', size=14, name='$f_L$')
add_sparse_markers(fig, _n0Range[_inRange], np.where(_fT_least, _fTpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='triangle-up-open',
                   color='blue', size=14, name='$f_T$')
add_sparse_markers(fig, _n0Range[_inRange], np.where(_fS_least, _fSpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='hash-open',
                   color='blue', size=14, name='$f_S$')
add_sparse_markers(fig, _n0Range[_inRange], np.where(_fH_least, _fHpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='hexagon-open',
                   color='blue', size=14, name='$f_H$')


add_sparse_markers(fig, _n0Range[_inRange], np.where(~_fL_least, _fL_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='line-ns',
                   color='red', size=14, name='$f_L$', showlegend=False, opacity=0.3)
add_sparse_markers(fig, _n0Range[_inRange], np.where(~_fT_least & ~_fL_least, _fTpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='triangle-up-open',
                   color='blue', size=14, name='$f_T$', showlegend=False, opacity=0.3)
add_sparse_markers(fig, _n0Range[_inRange], np.where(~_fS_least & ~_fL_least, _fSpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='hash-open',
                   color='blue', size=14, name='$f_S$', showlegend=False, opacity=0.3)
add_sparse_markers(fig, _n0Range[_inRange], np.where(~_fH_least & ~_fL_least, _fHpos_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid) - _f_max,
                   step=20, marker_symbol='hexagon-open',
                   color='blue', size=14, name='$f_H$', showlegend=False, opacity=0.3)

fig.add_trace(go.Scatter(
    x=[nf_LT[0]], y=[nf_LT[1]] - m*(nf_LT[0] - n0_mid) - _f_max,
    mode='markers',
    marker_symbol='star-open',
    marker=dict(color='magenta', size=16),
    name='LT-phase-transition',
))
fig.add_trace(go.Scatter(
    x=[nf_TS[0]], y=[nf_TS[1]] - m*(nf_TS[0] - n0_mid) - _f_max,
    mode='markers',
    marker_symbol='star-open',
    marker=dict(color='orange', size=16),
    name='TS-phase-transition',
))
fig.add_trace(go.Scatter(
    x=[nf_SH[0]], y=[nf_SH[1]] - m*(nf_SH[0] - n0_mid) - _f_max,
    mode='markers',
    marker_symbol='star-open',
    marker=dict(color='cyan', size=16),
    name='SH-phase-transition',
))
fig.add_trace(go.Scatter(
    x=[nf_HL[0]], y=[nf_HL[1]] - m*(nf_HL[0] - n0_mid) - _f_max,
    mode='markers',
    marker_symbol='star-open',
    marker=dict(color='black', size=16),
    name='HL-phase-transition',
))

fig.update_layout(
  height=600, width=800,
  title=dict(
      text=r'$\Large{\mathsf{Free\ energy\ vs\ Average\ Density\ }(\Delta\epsilon=' + f'{-0.50:.3f},' + r'g=' + f'{g_log:.3f}' + r')}$',
      font=dict(family="Arial", size=60, color="black")
  ),
  xaxis=dict(
      title=r'$\mathsf{Average\ Density\ } n_0$',
      titlefont=dict(family="Arial", size=16, color="black")
  ),
  yaxis=dict(
      title=r'$\mathsf{Free\ Energy\ Density\ } \Delta f \quad[\mathsf{slope\ adjusted}]$',
      titlefont=dict(family="Arial", size=16, color="black")
  ),
  font=dict(family="Arial", size=14, color="black"),  # global default
  plot_bgcolor="white",
  legend=dict(
      x=0.02,               # horizontal position (0 = left, 1 = right)
      y=0.98,               # vertical position (0 = bottom, 1 = top)
      xanchor='left',       # anchor legend's x to its left side
      yanchor='top',        # anchor legend's y to its top
      bgcolor='rgba(255,255,255,0.8)',  # semi-transparent background
      bordercolor='black',
      borderwidth=1,
      orientation='v'       # 'v' for vertical, 'h' for horizontal
  ),
)

light_gray = '#e6e6e6'  # subtle light gray (adjust hex to taste)
fig.update_xaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    linecolor='black', mirror=True, ticks='outside'
)
fig.update_yaxes(
    showgrid=True, gridcolor=light_gray, gridwidth=1,
    zeroline=True, zerolinecolor=light_gray,
    linecolor='black', mirror=True, ticks='outside'
)

fig.show()

In [ ]:
_n0Range[_inRange].shape

In [ ]:
np.where(_fL_least, _fL_np, np.nan)[_inRange] - m*(_n0Range[_inRange] - n0_mid)